In [1]:
import torch
from torch.utils.data import Dataset
import torchvision.transforms as transforms
import PIL

from custom_dataset import CustomDataset


# class CustomDataset(Dataset):
#     def __init__(self, image_path_to_class_dict, class_to_number_dict, apply_resize=False):
#         super().__init__()
#         self.image_path_to_class_dict = image_path_to_class_dict
#         self.class_to_number_dict = class_to_number_dict
#         self.image_paths = list(self.image_path_to_class_dict.keys())
#         self.resize = transforms.Resize((224, 224))  # Resize to 256x256 for ViT input
#         self.to_tensor = transforms.ToTensor()  # Converts images to tensor
#         self.normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                                               std=[0.229, 0.224, 0.225])  # Standard normalization for ViT
#         self.apply_resize = apply_resize

#     def __len__(self):
#         return len(self.image_paths)

#     def __getitem__(self, index):
#         # Open the image and convert to RGB
#         image = PIL.Image.open(self.image_paths[index]).convert('RGB')

#         # Apply resize if needed
#         if self.apply_resize:
#             image = self.resize(image)

#         # Convert the image to a PyTorch tensor and normalize
#         image = self.to_tensor(image)
#         image = self.normalize(image)

#         # Get the corresponding label
#         class_name = self.image_path_to_class_dict[self.image_paths[index]]
#         label = self.class_to_number_dict[class_name]

#         # Convert label to tensor
#         label = torch.tensor(label, dtype=torch.long)

#         return image, label
    
#     if __name__ == "__main__":
#         dataset = CustomDataset([1, 2, 3, 4, 5])
#         dataloader = DataLoader(dataset, num_workers=2) 
#         for batch in dataloader:
#             print(batch)

In [2]:
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms

our_data_simple = ImageFolder("train",
                              transform=transforms.Compose([transforms.Resize((256,256)), transforms.ToTensor()]),
                              target_transform=None)

class_to_number_dict = our_data_simple.class_to_idx

print(len(class_to_number_dict))

22


In [8]:
import torch.nn as nn
import timm
from torchvision.models import vit_b_16

class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        num_classes = len(class_to_number_dict)
        # Load the pre-trained ViT model
        self.vit_model = timm.create_model('vit_base_patch16_224', pretrained=True)

        # Replace the classification head (final layer) to match the number of classes
        self.vit_model.head = nn.Linear(self.vit_model.head.in_features, num_classes)

        # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

        self.vit_model.to(device)

    def forward(self, x):
        # Forward pass through the Vision Transformer model
        return self.vit_model(x)
    
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(device)

mps


In [ ]:
import os

import torch
from torch import nn
import torch.nn.functional as F
from torch.optim import Adam
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split

# parser = argparse.ArgumentParser(description= \
#                                      'scipt for training of project 2')
# parser.add_argument('--cuda', action='store_true', default=False,
#                     help='Used when there are cuda installed.')
# args = parser.parse_args()

# training process.
def train_net(net, trainloader, valloader):
########## ToDo: Your codes goes below #######
    val_accuracy = 0
    # val_accuracy is the validation accuracy of each epoch. You can save your model base on the best validation accuracy.

    # Define the loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(network.parameters(), lr=0.0001)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)


    # optimizer = Adam(network.parameters(), lr=0.001)

    num_epochs = 20
    best_val_acc = 0.0
    # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

    net.to(device)

    # Training loop
    # Implemented early stopping based on validation accuracy, saved the model at its best validation performance, and added a learning rate scheduler.
    for epoch in range(num_epochs):
        net.train()
        correct_train = 0
        total_train = 0
        running_loss = 0.0

        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = net(inputs)

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            _, predicted = torch.max(outputs.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()


            running_loss += loss.item()

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(trainloader)}")
        print(f"Training Accuracy: {100 * correct_train / total_train}%")

    train_accuracy = 100 * correct_train / total_train
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(trainloader)}, Training Accuracy: {train_accuracy}%")


    # Validation loop
    net.eval()
    correct_val = 0
    total_val = 0
    val_loss = 0.0
    with torch.no_grad():
        for data in valloader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = net(inputs)
            val_loss += criterion(outputs, labels).item()

            _, predicted = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_accuracy = 100 * correct_val / total_val
    print(f"Validation Accuracy: {val_accuracy}%")

    # Save the model if validation accuracy improves
    if val_accuracy > best_val_acc:
        best_val_acc = val_accuracy
        torch.save(net.state_dict(), 'best_model.pth')
        print(f"Best model saved with validation accuracy: {best_val_acc}%")

        # Step the scheduler
        scheduler.step()

    return best_val_acc
    # return val_accuracy

##############################################

############################################
# Transformation definition
# NOTE:
# Write the train_transform here. We recommend you use
# Normalization, RandomCrop and any other transform you think is useful.

# As we know that data augmentation helps improve the model's robustness. Used a wider variety of transformations, such as random rotation, color jitter, and affine transformations.

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalization
])


# train_transform = transforms.Compose([
#     transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
#     transforms.RandomHorizontalFlip(), # to be toggled for testing
#     transforms.ToTensor(),
#     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
# ])
####################################

####################################
# Define the training dataset and dataloader.
# You can make some modifications, e.g. batch_size, adding other hyperparameters, etc.

# Folder path containing the dataset (e.g., '../dataset/')
root = 'train'  # Replace with the path to your dataset

classes = os.listdir(root)

!rm -rf /train/.DS_Store

# register all images with their corresponding classes
image_path_to_class_dict = {}
for class_name in classes:
  images = os.listdir(os.path.join(root, class_name))
  for image in images:
    image_path_to_class_dict[os.path.join(root, class_name, image)] = class_name

our_data_simple = ImageFolder(root,
                              transform=transforms.Compose([transforms.Resize((256,256)), transforms.ToTensor()]),
                              target_transform=None)

class_to_number_dict = our_data_simple.class_to_idx

# Create an instance of the custom dataset
custom_dataset = CustomDataset(image_path_to_class_dict, class_to_number_dict, apply_resize=True)

# Split the dataset into train and validation sets (80% train, 20% val)
train_size = int(0.8 * len(custom_dataset))
val_size = len(custom_dataset) - train_size

trainset, valset = torch.utils.data.random_split(custom_dataset, [train_size, val_size])


# trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
#                                          shuffle=True, num_workers=4)
# valloader = torch.utils.data.DataLoader(valset, batch_size=32,
#                                          shuffle=True, num_workers=4)



trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                         shuffle=True, num_workers=4)
valloader = torch.utils.data.DataLoader(valset, batch_size=64,
                                         shuffle=False, num_workers=4)
####################################

# ==================================
# use cuda if called with '--cuda'.

network = Network()
# if args.cuda:
#     network = network.cuda()

# network = network.cuda()

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
network = network.to(device)


# train and eval your trained network
# you have to define your own
val_acc = train_net(network, trainloader, valloader)

print("final validation accuracy:", val_acc)

# ==================================

In [1]:
import os
import torch
from torch import nn
import torch.nn.functional as F
from torch.optim import Adam
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

# CustomDataset class needs to be defined before this
# Example: 
# class CustomDataset(torch.utils.data.Dataset):
#     pass  # Your custom dataset implementation

# Training process with additional tracking
def train_net(net, trainloader, valloader):
    # Define the loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(net.parameters(), lr=0.0001)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

    num_epochs = 20
    best_val_acc = 0.0
    prev_val_acc = 0.0  # Store the validation accuracy of the previous epoch
    net.to(device)

    # Training loop
    for epoch in range(num_epochs):
        net.train()
        correct_train = 0
        total_train = 0
        running_loss = 0.0

        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = net(inputs)

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            _, predicted = torch.max(outputs.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

            running_loss += loss.item()

        train_accuracy = 100 * correct_train / total_train
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(trainloader)}, Training Accuracy: {train_accuracy}%")

        # Validation loop
        net.eval()
        correct_val = 0
        total_val = 0
        val_loss = 0.0
        with torch.no_grad():
            for data in valloader:
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = net(inputs)
                val_loss += criterion(outputs, labels).item()

                _, predicted = torch.max(outputs.data, 1)
                total_val += labels.size(0)
                correct_val += (predicted == labels).sum().item()

        val_accuracy = 100 * correct_val / total_val
        print(f"Validation Accuracy: {val_accuracy}%")

        # Compare training and validation accuracy
        if val_accuracy < prev_val_acc and train_accuracy > prev_train_acc:
            print(f"Warning: Validation accuracy decreased while training accuracy increased at epoch {epoch+1}")

        prev_val_acc = val_accuracy  # Store current validation accuracy for the next comparison
        prev_train_acc = train_accuracy  # Store current training accuracy for the next comparison

        # Save the model if validation accuracy improves
        if val_accuracy > best_val_acc:
            best_val_acc = val_accuracy
            torch.save(net.state_dict(), 'best_model.pth')
            print(f"Best model saved with validation accuracy: {best_val_acc}%")

        # Step the scheduler
        scheduler.step()

    return best_val_acc


# ==================================
# Define the training dataset and dataloader.
root = 'train'  # Replace with the path to your dataset

# Folder path containing the dataset
classes = os.listdir(root)

# Remove hidden system files if present
!rm -rf /train/.DS_Store

# Custom dataset registration
image_path_to_class_dict = {}
for class_name in classes:
    images = os.listdir(os.path.join(root, class_name))
    for image in images:
        image_path_to_class_dict[os.path.join(root, class_name, image)] = class_name

# Create an instance of the custom dataset
custom_dataset = CustomDataset(image_path_to_class_dict, class_to_number_dict, apply_resize=True)

# Split the dataset into train and validation sets
train_size = int(0.8 * len(custom_dataset))
val_size = len(custom_dataset) - train_size
trainset, valset = torch.utils.data.random_split(custom_dataset, [train_size, val_size])

# Dataloaders for training and validation
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=4)
valloader = torch.utils.data.DataLoader(valset, batch_size=64, shuffle=False, num_workers=4)

# ==================================
# Initialize network and train
network = Network()
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
network = network.to(device)

# Train and evaluate
val_acc = train_net(network, trainloader, valloader)
print("Final validation accuracy:", val_acc)
# ==================================


NameError: name 'CustomDataset' is not defined